In [ ]:
from imports_LDVAE_local import * #imports_scgen_local have to be in same folder

In [ ]:
pre = "MH79"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
base_model_path_MH52,_,_,_ = h.return_local_paths(drive = drive,pre = 'MH52',add_path = False)

In [ ]:
model_dir_names = os.listdir(base_model_path_MH52)

In [ ]:
warnings.filterwarnings("ignore")
model_dict = {}
for model_name in model_dir_names:
    load_string = os.path.join(base_model_path_MH52,model_name) 
    model = scvi.model.LinearSCVI.load(load_string)
    model_dict[model_name.split('_')[3].split('.')[0]] = model

In [ ]:
celltypes = list(model_dict.keys())

In [ ]:
celltype = model_dir_names[0]
color_00hr = "#d7aca1"
color_06hr = "#34b6c6"
color_24hr ="#4063a3"
color_human = '#178F92'
color_cyno = "#094568"

color_cyno1 = "#134130"
color_cyno2 = "#8cae9e"
color_human1 = "#508ca7"
color_human2 = "#0f252f"

color_human = '#178F92'
color_cyno = "#094568"

palette_species = [color_human,color_cyno]
palette_timepoints = [color_00hr,color_06hr,color_24hr]
palette_individual = [color_cyno1,color_cyno2,color_human1,color_human2]

In [ ]:
main_folder_path = os.path.join(base_plots_path,"MH52")
if not os.path.exists(main_folder_path):
    os.mkdir(main_folder_path)

In [ ]:
for celltype in celltypes:
    adata = model_dict[celltype].adata
    adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    adata.obs['scvi_clusters'] = adata.obs['scvi_clusters'].astype('category')
    h.prepare_umap(adata)
    
    # Define a list of configurations for the subplots
    plot_configs = [
        {"color": "timepoint", "palette": palette_timepoints, "title": "Timepoint"},
        {"color": "species", "palette": palette_species, "title": "Species"},
        {"color": "scvi_clusters", "palette": None, "title": "Clusters"},
        {"color": "individual", "palette": palette_individual, "title": "Individual"}
    ]
    
    # Create a figure with subplots
    fig, axes = plt.subplots(1, len(plot_configs), figsize=(20, 5), constrained_layout=True)
    fig.suptitle(f"UMAP {celltype}", fontsize=16)

    # Generate plots for each configuration
    for ax, config in zip(axes, plot_configs):
        h.umap_plot_palette_with_ax(
            adata=adata,
            color=config["color"],
            base_plots_path=None,  # Skip saving individual plots
            plot_name=None,        # Skip saving individual plots
            show_in_nb=False,
            frameon=False,
            palette=config["palette"],
            ax=ax  # Pass the subplot axis
        )
        ax.set_title(config["title"])
    
    # Show the combined plot in the notebook
    plt.savefig(os.path.join(main_folder_path,pre + 'UMAPs_GE_space_' + str(celltype) + '.pdf'),
                dpi=300,
                bbox_inches = 'tight')
    plt.show()

In [ ]:
# Define the number of cell types and configurations
num_celltypes = len(model_dir_names)  # 8 in this case
num_configs = 4  # timepoint, species, clusters, individual


fig, axes = plt.subplots(num_celltypes, num_configs, figsize=(20, 3 * num_celltypes), constrained_layout=True)
#fig.suptitle("UMAPs for All Cell Types", fontsize=20)

# Iterate through cell types and their corresponding rows
for row_idx, celltype in enumerate(celltypes):
    adata = model_dict[celltype].adata
    adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    adata.obs['scvi_clusters'] = adata.obs['scvi_clusters'].astype('category')
    h.prepare_umap(adata)
    
    # Define configurations for each column
    plot_configs = [
        {"color": "timepoint", "palette": palette_timepoints, "title": "timepoint"},
        {"color": "species", "palette": palette_species, "title": "species"},
        {"color": "scvi_clusters", "palette": None, "title": "clusters"},
        {"color": "individual", "palette": palette_individual, "title": "individual"}
    ]
    
    # Generate plots for the current cell type
    for col_idx, config in enumerate(plot_configs):
        ax = axes[row_idx, col_idx]  # Select subplot
        h.umap_plot_palette_with_ax(
            adata=adata,
            color=config["color"],
            base_plots_path=None,  # Skip saving individual plots
            plot_name=None,        # Skip saving individual plots
            show_in_nb=False,
            frameon=False,
            palette=config["palette"],
            ax=ax  # Pass the subplot axis
        )
        # Set titles for the first row and row labels
        if row_idx == 0:
            ax.set_title(config["title"], fontsize=14)
        if col_idx == 0:
            ax.set_ylabel(celltype, fontsize=12, rotation=90, labelpad=1)
plt.tight_layout()
# Save the entire figure as a PDF
plt.savefig(os.path.join(main_folder_path, pre + 'UMAPs_GE_space_all_celltypes.pdf'),
            dpi=300,
            bbox_inches='tight')
plt.show()

## Latent space

In [ ]:
# Define the number of cell types and configurations
num_celltypes = len(model_dir_names)  # Number of rows
num_configs = 4  # timepoint, species, clusters, individual

# Create the figure
fig, axes = plt.subplots(num_celltypes, num_configs, figsize=(20, 2.5 * num_celltypes), constrained_layout=True)
#fig.suptitle("Latent UMAPs for All Cell Types", fontsize=20)

# Iterate through cell types and plot configurations
for row_idx, celltype in enumerate(celltypes):
    adata = model_dict[celltype].adata
    adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    adata.obs['scvi_clusters'] = adata.obs['scvi_clusters'].astype('category')

    # Get latent representation and prepare UMAP
    model = model_dict[celltype]
    latent_adata = h.latent_embedding_LDVAE(network=model,adata=model.adata)
    h.prepare_umap(latent_adata)

    # Define configurations for each column
    plot_configs = [
        {"color": "timepoint", "palette": palette_timepoints, "title": "timepoint"},
        {"color": "species", "palette": palette_species, "title": "species"},
        {"color": "scvi_clusters", "palette": None, "title": "clusters"},
        {"color": "individual", "palette": palette_individual, "title": "individual"}
    ]

    # Generate subplots for this cell type
    for col_idx, config in enumerate(plot_configs):
        ax = axes[row_idx, col_idx]  # Select the subplot
        h.umap_plot_palette_with_ax(
            adata=latent_adata,
            color=config["color"],
            base_plots_path=None,  # Skip saving individual plots
            plot_name=None,        # Skip saving individual plots
            show_in_nb=False,
            frameon=False,
            palette=config["palette"],
            ax=ax
        )
        # Set the title for the first row and row labels
        if row_idx == 0:
            ax.set_title(config["title"], fontsize=14)
        if col_idx == 0:
            ax.set_ylabel(celltype, fontsize=12, rotation=90, labelpad=10)

plt.savefig(os.path.join(main_folder_path, pre + 'UMAPs_latent_space_all_celltypes.pdf'),
            dpi=300,
            bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()